In [ ]:
!c:/Python38/python.exe -m pip install tensorflow

# Olhos

## Código criado para utilização em visão computacional para reconhecimento de cones de transito


### Preparo

In [ ]:
import cv2, numpy as np
from tensorflow import keras
from keras.models import load_model
dado = "./others/train/keras_model.h5"

### Instanciando a Machine

In [ ]:
video = cv2.VideoCapture(0, cv2.CAP_DSHOW)
model = load_model(dado, compile = False)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
classes = ["cones"]

class neuronium():

    def preProcess(img):
        imgPre = cv2.GaussianBlur(img, (5,5), 3)
        imgPre = cv2.Canny(imgPre, 90, 140)
        kernel = np.ones((4,4), np.uint8)
        imgPre = cv2.dilate(imgPre, kernel, iterations=3)
        imgPre = cv2.erode(imgPre, kernel, iterations=3)
        return imgPre 
    def recogCones(img):
        imgCone = cv2.resize(img, (224, 224))
        imgCone = np.asarray(imgCone)
        imgConeNormalize = (imgCone.astype(np.float32)/127.0)-1
        data[0] = imgConeNormalize
        prediction = model.predict(data)
        index = np.argmax(prediction)
        trustPercent = prediction[0][index]
        classe = classes[index]
        return classe,trustPercent  
    
    while True:
        _,img = video.read()
        try:
            img = cv2.resize(img, (640, 480))
        except:
            break

        imgPre = preProcess(img)
        contours, hi = cv2.findContours(imgPre, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 2000:
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
                cut = img[y:y +h, x:x +w]
                classe, conf = recogCones(cut)
                cv2.putText(img, str(classe), (x,y), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,255,255), 2 )
        
        cv2.imshow('IMG', img)
        cv2.imshow('IMG PrePro', imgPre)
        cv2.waitKey(1)